In [26]:
from sys_data import System_Data
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [27]:
system_data = System_Data(file_name='Case_33BW_Data.xlsx')
model = gp.Model("Expert_Model")

NT, N_Branch, N_TL, N_NL, N_Bus, pIn, N_DG, DG_Mask, R_Branch, X_Branch, Big_M_V, V0, \
    V_min, V_max, Pd, Qd, S_Branch, P_DG_min, P_DG_max, Q_DG_min, Q_DG_max, BigM_SC, BSDG_Mask, \
    Big_M_FF = system_data.args_expert

In [28]:
PF = model.addMVar(shape=(N_Branch,NT), vtype=GRB.BINARY, name="PF")
QF = model.addMVar(shape=(N_Branch,NT), vtype=GRB.BINARY, name="QF")
V = model.addMVar(shape=(N_Bus,NT), vtype=GRB.CONTINUOUS, name="V")

P_dg = model.addMVar(shape=(N_DG,NT), vtype=GRB.CONTINUOUS, name="P_dg")
Q_dg = model.addMVar(shape=(N_DG,NT), vtype=GRB.CONTINUOUS, name="Q_dg")

In [29]:
# ------------------潮流--------------------
# 1. Bus PQ Blance: S_jk - S_ij = S_inj
model.addConstr(pIn @ PF == DG_Mask @ P_dg - Pd)
model.addConstr(pIn @ QF == DG_Mask @ Q_dg - Qd)

  # 2. Voltage : U_j - U_i = r*Q_ij + x*P_ij
# model.addConstr(pIn.T @ V == R_Branch * PF + X_Branch * QF)
# model.addConstr(V[0,:] == V0)
# model.addConstr(V[1:,:] <= V_max)
# model.addConstr(V[1:,:] >= V_min)

    # % 4. 线路
model.addConstr(PF >= -S_Branch)
model.addConstr(PF <= S_Branch)
model.addConstr(QF >= -S_Branch)
model.addConstr(QF <= S_Branch)
  # ------------DG ----------------
model.addConstr(P_dg >= P_DG_min)
model.addConstr(P_dg <= P_DG_max)
model.addConstr(Q_dg >= Q_DG_min)
model.addConstr(Q_dg <= Q_DG_max)

model.setObjective(0)

In [30]:
model.Params.OutputFlag = 1
model.optimize()
print(V.X)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i5-1340P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1210 rows, 605 columns and 1690 nonzeros
Model fingerprint: 0xe23b6a2d
Variable types: 235 continuous, 370 integer (370 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-04, 1e+00]
Presolve removed 440 rows and 165 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


GurobiError: Unable to retrieve attribute 'X'

In [ ]:
Big_M_V * (1 - b)